In [1]:
#temp

#Attempt at creating a functional rope bot for 312th squadron, no.1

import discord
import datetime
from discord.ext import commands, tasks
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from apscheduler.triggers.cron import CronTrigger
import random as rd


utc = datetime.timezone.utc
time = datetime.time(tzinfo=utc)
#hour minute second for when to await post accountability register
global hr
global mn
hr = 18
mn = 0
#same but for weekend accountability register
global wk_hr
global wk_mn
wk_hr = 17
wk_mn = 0
channel_id = 1079811783231033344
msg_id_list = []
wk_msg_list = []
month_msg_list = []
edited_list = []
clean_list = []


async def post(message_str):
    
    channel = client.get_channel(channel_id)
    sent = await channel.send(message_str)
    clean_list.append(sent.id)
    
async def post_acc(message_str):
    
    channel = client.get_channel(channel_id)
    sent = await channel.send(message_str)
    clean_list.append(sent.id)
    msg_id_list.append(sent.id)
    
async def post_wk(message_str):
    
    channel = client.get_channel(channel_id)
    sent = await channel.send(message_str)
    wk_msg_list.append(sent.id)
    msg_id_list.append(sent.id)
    
async def post_monthly(message_str):
    
    channel = client.get_channel(channel_id)
    sent = await channel.send(message_str)
    clean_list.append(sent.id)
    msg_id_list.append(sent.id)

async def acc_func():
    
    await acc_clean_up()
    channel = client.get_channel(channel_id)
    
    msg_list = ["Greetings Airmen Leaders, here is your sheet: react to signup", "0525 Accountability -> Phase 1 A-L: ", "0525 Accountability -> Phase 1 M-Z: ",
    "0525 Accountability -> Phase 2 A-L: ", "0525 Accountability -> Phase 2 M-Z: ",
    "0525 Accountability -> Phase 3 A-L: ", "0525 Accountability -> Phase 3 M-Z: "]
    
    
    for each in msg_list:     
        sent = await post_acc(each)
        
    
async def wk_func():
    
    await wk_clean_up()
    channel = client.get_channel(channel_id)
    
    msg_list = ["Greetings Airmen Leaders, here is your sheet: react to signup", "2145 Accountability -> Phase 1 A-L: ", "2145 Accountability -> Phase 1 M-Z: ",
    "2345 Accountability -> Phase 2 A-L: ", "2345 Accountability -> Phase 2 M-Z: "]
    
    for each in msg_list:     
        sent = await post_wk(each)

async def msg_response(msg_str, channel, user_id):
    
    content = msg_str.lower()
    name_cont = msg_str
    cont_split = content.split(" ")
    name_split = name_cont.split(" ")
    binary = await check_auth(user_id)
    list_of_valid_options = ["firedog", "spinstra", "accountability", "cleanup", "openranks", "meeting"]

    #Generic Help Section
    if(cont_split[0] == "bothelp"):
        if(cont_split[1] == "firedog"):
            await post("""
    HUA firedog, this bot is designed to post accountability messages that Airmen Leaders can react to for the purposes of organization amongst Airmen Leaders. If all is going well, the bot will add the name of the reacting AL to the message, and it will be easy to see who is doing what on a daily basis. In the event of the bot no longer working as intended, it's suggested that you try being nice to a Spinstra student since it's highly unlikely you'll able able to fix it.""")
        if(cont_split[1] == "spinstra"):
            await post("""
    Welcome Professor Falken, as someone who should be capable of programming, and also as someone who probably has programming in their fundamental course for 9S100, you can fix the bot yourself. Source code: https://github.com/twm77bot/NopeRopeBot In the event that you can't manage it, or you're desperate, feel free to contact the author Mitchell; via email -> talon.mitchell.1@us.af.mil """)
        if cont_split[1] not in list_of_valid_options:
            sent = await post("https://tenor.com/31lr.gif")
            sent = await post("Bot Help Option not in list of Valid Options. Try again or contact Admin.")
        if(cont_split[1] == "accountability"):
            await post("Please enter a new time for accountability registration message to be posted by the Bot. Format will be as follows, follow it exactly; \"hour minute\" . Note, these must be in integer form and any number less than 10 cannot be prefixed by a zero (So 09 is WRONG)")
            await post("Full Example Here -> accountability set 4 0 (4am on the dot, no minutes)")
        if(cont_split[1] == "cleanup"):
            await clean_up()
            #await acc_clean_up()
            #await wk_clean_up()
        if(cont_split[1] == "openranks"):
            await post("Please enter a new day & time for accountability registration message to be posted by the Bot. Format will be as follows, follow it exactly \"day hour minute second\". Note, these be in integer form and any number less than 10 cannot be prefixed by a zero")
            await post("Full Example Here -> openranks set 1 4 0 (first day of the month at 4:00 am)")
        if(cont_split[1] == "meeting"):
            await post("Please enter a new time for the bot to post a Peer AL Meeting announcement. Note: The time to post should be 2 hours before the actual meeting time")
            await post("Full Example Here -> meeting set Saturday 11 0  (this will set the announcement to post on saturday @ 11:00, with the actual meeting time at 1pm that day)")
            
    #variable change via discord commands, weekday and weekend accountability
    if(cont_split[0] == "accountability"):
        if(cont_split[1] == "set") and binary:
            try:
                temp_hr = int(cont_split[2])
                temp_mn = int(cont_split[3])
                if(temp_hr>24) or (temp_hr<0) or (temp_mn>59) or (temp_mn<0):
                    await post("https://tenor.com/RZBs.gif")
                    await post("Time in unacceptable format, try again or contact Admin")
                else:
                    await post("New Accountability posting Time accepted: " +str(temp_hr)+ ":" +str(temp_mn))
                    hr = temp_hr
                    mn = temp_mn
                    #await update_scheduler(temp_hr, temp_mn, temp_sc)
            except:
                sent = await post("Exception in Accountability Set Function, please inform Admin.")
            finally:
                await update_scheduler(temp_hr, temp_mn)
        elif(cont_split[1] == "help"):
            await post("This command changes the time the bot will post the sign-up sheet for daily (morning accountability) on the preceding day")
            await post("Example of correct command format -> accountability set 19 0  (this will set the post time to 19:00)")            
        else:
            sent = await post("https://tenor.com/vIwH.gif")
            sent = await post("Unacceptable option for accountability, or user unauthorized, try again or contact Admin")
    if(cont_split[0] == "weekend"):
        if(cont_split[1] == "set") and binary:
            try:
                temp_hr = int(cont_split[2])
                temp_mn = int(cont_split[3])
                if(temp_hr>24) or (temp_hr<0) or (temp_mn>59) or (temp_mn<0):
                    sent = await post("https://tenor.com/bM6e7.gif")
                    await post("Time in unacceptable format, try again or contact Admin")
                else:
                    await post("New Accountability Weekened Posting Time accepted: " +str(temp_hr)+ ":" +str(temp_mn))
                    wk_hr = temp_hr
                    wk_mn = temp_hr
                    #await update_wknd_scheduler(temp_hr, temp_mn, temp_sc)
            except:
                sent = await post("Exception in Weekend Set Function, please inform Admin")
            finally:
                await update_wknd_scheduler(temp_hr, temp_mn)
        elif(cont_split[1] == "help"):
            await post("This command changes the time the pot will post the weekend sign-up sheet for weekend accountability")
            await post("Example of correct command format -> weekend set 16 0  (this will set the post time to 16:00)")
        else:
            sent = await post("https://tenor.com/vIwH.gif")
            await post("Unacceptable option for weekend, try again or contact Admin")
    if(cont_split[0] == "openranks"):
        if(cont_split[1] == "set") and binary:
            try:
                dy = int(cont_split[2])
                hr = int(cont_split[3])
                mn = int(cont_split[4])
                if(dy<1) or (dy>31) or (hr<0) or (hr>24) or (mn<0) or (mn>60):
                    sent = await post("https://tenor.com/bif1X.gif")
                    await post("Time in unacceptable format, try again or contact Admin")
                else:
                    await post("New Monthly post Time accepted: Day of Month -> " +str(dy)+ " @ " +str(hr)+ ":" +str(mn))
            except:
                sent = await post("Exception in Openranks Set Function, please inform Admin")
            finally:
                await update_monthly_scheduler(dy, hr, mn)
        elif(cont_split[1] == "help"):
            sent = await post("This command changes the await posting slot for the open-ranks register. Format is as follows: day of month hour minute. Example 1 4 0 - this would be the first day of the month at 4am on the dot.")
        else:
            sent = await post("https://tenor.com/RZBs.gif")
            await post("Unacceptable option for openranks, or user unauthorized, try again or contact Admin")
    if(cont_split[0] == "meeting"):
        if(cont_split[1] == "set") and binary:
            try:
                dy = day_conv(cont_split[2])
                hr = int(cont_split[3])
                mn = int(cont_split[4])
                if(dy<1) or (dy>31) or (hr<0) or (hr>24) or (mn<0) or (mn>60):
                    sent = await post("https://tenor.com/MUNo.gif")
                    await post("Time in unacceptable format, try again or contact Admin")
                else:
                    await post("New Meeting Announcement Post Time accepted: " +day_fix(dy)+ " " +str(hr)+ ":" +str(mn))
            except:
                sent = await post("Exception in Meeting Set Function, please inform Admin")
            finally:
                await update_monthly_scheduler(dy, hr, m)
        elif(cont_split[1] == "help"):
            await post("This command changes the time the bot will post the peer AL meeting announcement")
            await post("Example of correct command format -> meeting set 6 10 0  (this will set the post time to 10:00 on Sunday)")
        else:
            await post("https://tenor.com/oZxZ.gif")
            await post("Unacceptable option for meeting, or unauthorized user, try again or contact Admin")
            
    if(cont_split[0] == auth_pass):
        await new_auth(user_id)
        await pass_gen()
        
    if(cont_split[0] == "deauthorize"):
        print(" ".join(name_split[1:]))
        if binary:
            await del_auth(" ".join(name_split[1:]))
        else:
            await post("You are not authorized to use this command.")
        
async def clean_up():
    
    channel = client.get_channel(channel_id)
    
    for each in clean_list:
        msg = await channel.fetch_message(each)
        await msg.delete()
        
async def acc_clean_up():
    
    channel = client.get_channel(channel_id)
    
    for each in msg_id_list:
        msg = await channel.fetch_message(each)
        await msg.delete()

async def wk_clean_up():
    
    channel = client.get_channel(channel_id)
    
    for each in wk_msg_list:
        msg = await channel.fetch_message(each)
        await msg.delete()
        
async def monthly_acc():
    
    channel = client.get_channel(channel_id)
    monthly_list = ["Greetings Airmen Leaders, here is your sheet: react to signup",
                 "Wk1 OR Accountability Phase 1 A-L: ", "Wk1 OR Accountability Phase 1 M-Z: ",
                 "Wk1 OR Accountability Phase 2 A-L: ", "Wk1 OR Accountability Phase 2 M-Z: ",
                 "Wk2 OR Accountability Phase 3 NWing A-L: ", "Wk2 OR Accountability Phase 3 NWing M-Z: ",
                 "Wk3 OR Accountability Phase 1 A-L: ", "Wk3 OR Accountability Phase 1 M-Z: ",
                 "Wk3 OR Accountability Phase 2 A-L: ", "Wk3 OR Accountability Phase 2 M-Z: ",
                 "Wk4 OR Accountability Phase 3 SWing A-L: ", "Wk4 OR Accountability Phase 3 SWing M-Z: ",
                 "Wk 1 Inpro Pickup Slot: ", "Wk 1 Inpro Pickup Slot: ",
                 "Wk 2 Inpro Pickup Slot: ", "Wk 2 Inpro Pickup Slot: ",
                 "Wk 3 Inpro Pickup Slot: ", "Wk 3 Inpro Pickup Slot: ",
                 "Wk 4 Inpro Pickup Slot: ", "Wk 4 Inpro Pickup Slot: "]
    for each in monthly_list:
        sent = await post_monthly(each)
        
async def meeting_post():
    
    channel = client.get_channel(channel_id)
    msg = "Greetings Airmen Leaders, this week's peer meeting is scheduled to take place in 2 hours. Please attend or inform senior AL about your absence."
    sent = await post(msg)
    
async def pass_gen():
    
    material =  "abcdefghijklmnopqrstuvwxyz1234567890!@#$%^&*"
    source = list(material)
    pass_list = []
    while(len(pass_list)<8):
        x = rd.choice(source)
        pass_list.append(x)
    global auth_pass
    auth_pass = "".join(pass_list)
    print(auth_pass)
    
async def new_auth(user_id):

    check = []
    with open("dbot_authorized_users.csv", "r") as f:
        data = f.readlines()
        for each in data:
            check.append(each)
    if str(user_id) in check:
        await post("This User is already authorized")
    else:
        with open("dbot_authorized_users.csv", "a") as f:
            f.writelines([str(user_id)])
        await post("New Authorized User Added")
    
        
async def del_auth(user_name):
    
    check = []
    with open("dbot_authorized_users.csv", "r") as f:
        data = f.readlines()
        for each in data:
            check.append(each)
    guild = client.get_guild(1079790688088100984)
    person = guild.get_member_named(user_name)
    comp = person.id
    print(comp)
    if str(comp) in check:
        check.remove(str(comp))
        await post("User has been removed from list of authorized")
        with open("dbot_authorized_users.csv", "w") as f:
            for each in check:
                f.writelines([str(each)])
    else:
        await post("User not in list of authorized, cannot remove")
            
async def check_auth(user_id):
    
    check = []
    with open("dbot_authorized_users.csv", "r") as f:
        data = f.readlines()
        for each in data:
            check.append(each)
    if str(user_id) in check:
        return True
    else:
        return False

class MyClient(discord.Client):

                
    async def on_message(self, message):
        #print(f'Message from {message.author}: {message.content}')
        channel = client.get_channel(channel_id)
        author = message.author
        user_id = author.id
        await msg_response(str(message.content), channel, user_id)
        
    async def on_raw_reaction_add(self, payload: discord.RawReactionActionEvent):
        #print(payload.emoji.name)
        channel = client.get_channel(channel_id)
        #await post(payload.emoji.name)
        id_check = payload.message_id
        if id_check in msg_id_list:
            if id_check in edited_list:
                pass
            else:
                guild = client.get_guild(1079790688088100984)
                filling = await channel.fetch_message(id_check)
                old = filling.content
                person = guild.get_member(payload.user_id)
                name = person.nick
                new = old + " " + name
                await filling.edit(content=new)
                edited_list.append(id_check)
        
    

    
    async def on_ready(self):
        print(f'Logged on as {self.user}!')
        
        global scheduler
        scheduler = AsyncIOScheduler()
        global monday
        global tuesday
        global wednesday
        global thursday
        global friday
        global friday_wknd
        global saturday_wknd
        global monthly
        global meeting
        
        
        #Mon-Fri Default values
        monday = scheduler.add_job(acc_func, CronTrigger(day_of_week=6, hour=hr, minute=mn))
        tuesday = scheduler.add_job(acc_func, CronTrigger(day_of_week=0, hour=hr, minute=mn))
        wednesday = scheduler.add_job(acc_func, CronTrigger(day_of_week=1, hour=hr, minute=mn))
        thursday = scheduler.add_job(acc_func, CronTrigger(day_of_week=2, hour=hr, minute=mn))
        friday = scheduler.add_job(acc_func, CronTrigger(day_of_week=3, hour=hr, minute=mn))
        
        
        #Friday Night and Saturday Night default values
        friday_wknd = scheduler.add_job(wk_func, CronTrigger(day_of_week=4, hour=wk_hr, minute=wk_mn))
        saturday_wknd = scheduler.add_job(wk_func, CronTrigger(day_of_week=4, hour=wk_hr, minute=wk_mn))
         
          #clean up functions, cleans up accountability msgs on friday, and weekend stuff monday
        #acc_cleaner = scheduler.add_job(acc_clean_up, CronTrigger(day_of_week=4, hour=9, minute=0, second=0))
        #wknd_cleaner = scheduler.add_job(wk_clean_up, CronTrigger(day_of_week=0, hour=0, minute=0, second=0))
        
        #Monthly Acc Stuff
        monthly = scheduler.add_job(monthly_acc, CronTrigger(month='1-12', day=1, hour=0, minute=0))
        
        #Meeting Post time
        meeting = scheduler.add_job(meeting_post, CronTrigger(day_of_week=6, hour=11, minute=0))
        
        #await update_scheduler()
        
        #pass generator time
        pass_gen_go = scheduler.add_job(pass_gen, CronTrigger(day_of_week='0-6', hour=11, minute=41))
        scheduler.start()
        #await monthly_acc()
        await pass_gen()
        
async def update_scheduler(hr, mn):
    
    monday.reschedule(CronTrigger(day_of_week=6, hour=hr, minute=mn))
    tuesday.reschedule(CronTrigger(day_of_week=0, hour=hr, minute=mn))
    wednesday.reschedule(CronTrigger(day_of_week=1, hour=hr, minute=mn))
    thursday.reschedule(CronTrigger(day_of_week=2, hour=hr, minute=mn))
    friday.reschedule(CronTrigger(day_of_week=3, hour=hr, minute=mn))
    
async def update_wknd_scheduler(hr, mn):
    
    friday_wknd.reschedule(CronTrigger(day_of_week=4, hour=hr, minute=mn))
    saturday_wknd.reschedule(CronTrigger(day_of_week=5, hour=hr, minute=mn))
    
async def update_monthly_scheduler(dy, hr, mn):
    
    monthly.reschedule(CronTrigger(month='1-12', day=dy, hour=hr, minute=mn))
    print("Update monthly called")
    print(dy)
    print(hr)
    print(mn)
    
async def update_meeting_scheduler(dy, hr, mn):
    
    meeting.reschedule(CronTrigger(day_of_week=dy, hour=hr, minute=mn))
    
def day_fix(x):
    
    days =["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return days[x]

def day_conv(day_string):
    
    days =["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    try:
        index = days.index(day_string)
        return index
    except:
        pass

intents = discord.Intents.default()
intents.message_content = True
intents.members = True

client = MyClient(intents=intents)
client.run('TOKEN')




ModuleNotFoundError: No module named 'discord'